## This Notebook will create a GMM to classify shots into one of the 6 actions

In [1]:
import PlottingFunction as pf
import DataManipulation as dm
import matplotlib.pyplot as plt
import Policy_iteration_functions as f
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal
import winsound

%matplotlib qt5

In [5]:
df  = pd.read_pickle(Path.cwd() / 'pickle' / 'TP' / 'df_double_prime.plk')
df = df[df.second_error != 'none']
df = df.reset_index(drop=True)
df = dm.add_landing_region_to_df(df, more_actions = True)

In [6]:
proportions = df.groupby('region').count().iloc[:,0]
total_rally = proportions['OBDrop':'deuceShortSide'].sum()
weights_rally = proportions['OBDrop':'deuceShortSide']/total_rally
total_deuce_serve = proportions['serve_deuce_corner':'serve_deuce_middle'].sum()
weights_deuce_serve = proportions['serve_deuce_corner':'serve_deuce_middle']/total_deuce_serve
total_ad_serve = proportions['serve_ad_corner':'serve_ad_middle'].sum()
weights_ad_serve = proportions['serve_ad_corner':'serve_ad_middle']/total_ad_serve

del df

In [4]:
region_labels = proportions.index.values
df_region_dict = {}
distribution_region_dict = {}

eps = 5

for region_label in region_labels:
    # slice for particular region/action
    df_region = df[df.region == region_label]
    # check if theres more than 20,000, if so, sample just 20,000
    if df_region.shape[0] > 20000:
        df_region = df_region.sample(20000, random_state=0)
    
    # save the subset of shots
    df_region_dict[region_label] = df_region
    
    # fit and save the gaussian distribution
    X = df_region[['x02', 'y02']]
    X = X.to_numpy()
    
    mean = np.mean(X, axis=0)
    cov = np.cov(X, rowvar=0)
    # Add epsilon error
    cov *= eps
    
    distribution = multivariate_normal(mean=mean, cov=cov)
    distribution_region_dict[region_label] = distribution

# del df

In [5]:
# x, y = np.mgrid[0:16:0.1, -7:7:0.1]
# pos = np.dstack((x, y))
# zShort = 0; zDeep = 0
# 
# for label, distribution in distribution_region_dict.items():
#     if 'serve' not in label:
#         if ('Deep' in label) or ("Base" in label):
#             zDeep += distribution.pdf(pos)
#         elif 'Corner' not in label:
#             zShort += distribution.pdf(pos)
#     
# fig = plt.figure()
# plt.subplot(2,1,1)
# plt.contourf(x, y, zShort,zorder=2)
# pf.createCourt(plt.gca(), compressed=True, only_half=False, big_buckets=True)
# plt.subplot(2, 1, 2)
# plt.contourf(x, y, zDeep,zorder=2)
# pf.createCourt(plt.gca(), compressed=True, only_half=False, big_buckets=True)
# 

In [6]:
# cmaps = ["Blues", "Reds", "Oranges", "Greens", "Purples", "Greys", "Blues", "Reds", "Oranges", "Greens", "Purples", "Greys"]
# 
# plt.figure()
# i = 0
# for label, df_region in df_region_dict.items():    
#     if 'serve' not in label:
#         if ('Deep' in label) or ("Base" in label):
#             X = df_region[['x02', 'y02']]
#             X = X.to_numpy()
#             # pf.createKde(X[:,0],X[:,1],compressed=True, only_half=False,big_buckets=True, n_levels=15, cmap=cmaps[i])
#             # i = i+1
#         elif 'Corner' not in label:
#             if "OBDrop" not in label:
#                 X = df_region[['x02', 'y02']]
#                 X = X.to_numpy()
#                 pf.createKde(X[:,0],X[:,1],compressed=True, only_half=False,big_buckets=True, n_levels=15, cmap=cmaps[i])
#                 i = i+1

In [7]:
df = dm.actionProbabilityVector(df,distribution_region_dict, region_labels, weights_rally, weights_deuce_serve, weights_ad_serve)

# probableAction = []
# for index, shot in df.iterrows(): # loop through the rows for the particular state
#     probableAction.append(np.random.choice(region_labels, p = shot['df_serve_deuce_middle':]))
# 
# df['probableAction'] = probableAction

# df.to_pickle('df_probable_actions_cov_eps.plk')

In [8]:
# df  = pd.read_pickle(Path.cwd() / 'pickle' /  'df_probable_actions.plk')

In [9]:
# df_compare = df.groupby(["region", "probableAction"]).size().reset_index(name="Time")
# df_compare = df_compare[df_compare.region != df_compare.probableAction]
# percent_diff = df_compare['Time'].sum()/871869*100
# winsound.Beep(2000,800)
# percent_diff

In [10]:
shot = 245642
# shot = 255937
# shot = 272631

x, y = np.mgrid[0:16:0.1, -7:7:0.1]
pos = np.dstack((x, y))
zDeep = 0

for label, distribution in distribution_region_dict.items():
    if 'serve' not in label:
        if ('Deep' in label) or ("Base" in label):
            zDeep += distribution.pdf(pos)
    
fig = plt.figure()
plt.subplot(111)
plt.contourf(x, y, zDeep,zorder=2)
pf.createCourt(plt.gca(), compressed=True, only_half=False, big_buckets=True)

df_sample = df.iloc[shot:shot+1,:]
pf.plot1D(df_sample,compressed=True, only_half=False,big_buckets=True,)

del df

In [4]:
df = df[df.Type == 'rally']
df = df.sample(100000)
plt.figure()
pf.plot1D(df,compressed=True, only_half=False,big_buckets=True,)
del df